# Séries temporelles avec Pandas

L'analyse temporelle est une partie essentielle de l'analyse de données. 

Dans cette partie vous verrez : 

- L'objet Datetime
- Quelles sont les fonctions de traitement de dates sous Pandas
- Les fonctions d'aggrégation temporelles

Pandas dispose de plusieurs méthode pour gérer et manipuler des données temporelles :

## Datetime (Python)

Le module **Datetime** en Python permet de réaliser plusieurs opérations avec des dates : 

In [3]:
from datetime import datetime

now = datetime.now()

l'objet **Datetime** stocke la date et l'heure jusqu'à la microsecond, jugez plutôt : 

In [ ]:
now 

Nous pouvons accéder à différentes à l'année, le mois, le jour etc.. 

In [ ]:
print now.year
print now.month
print now.day

## String --> datetime

Bien entendu, les données temporelles sont rarement stockées au format dates. Elles sont souvent stockées et parsées sous forme de texte

La méthode **strptime** du module **datetime** de Python permet de transformer un String en datetime en y intégrant un **format de date**

In [4]:
chaine = '2013-02-13'

In [5]:
datetime.strptime(chaine,'%Y-%m-%d')

datetime.datetime(2013, 2, 13, 0, 0)

On peut retrouver l'intégralité des formats de date ici - https://docs.python.org/2/library/datetime.html#strftime-and-strptime-behavior

In [7]:
import pandas as pd

Pandas permet de parse facilement les données d'une Série grâce à la méthode **to_datetime** :

In [ ]:
datestring = ['2013-04-03', '2013-04-04', '2013-04-05', '2013-04-06']

In [ ]:
pd.to_datetime(datestring)

Remarquons que la **liste** datestring a été transformée en objet Pandas **tseries.index.DatetimeIndex** 

Plusieurs options (qui dépasse ce focus) sont également à noter : la longueur de la série, la fréquence, la timezone etc..

## Time et DataFrame 

Lorsque vous disposez d'un set de données contenant des données temporelles, il advient de bien indexer en utilisant l'objet  **DatetimeIndex**. 

Prenons par exemple une nouvelle liste de dates (format chaine) : 

In [ ]:
datestr = ['2013-04-03', '2013-04-04', '2013-04-05', '2013-05-06','2014-06-07','2014-07-08']

Nous transformons cette liste de String en un objet DatetimeIndex en utilisant la méthode **to_datetime** :

In [ ]:
dates = pd.to_datetime(datestr)

Si nous souhaitons maintenant créer un DataFrame, nous pouvons spécifier, au moment de la création, l'index à utiliser grâce au paramètre **index** :

In [ ]:
df = pd.DataFrame(np.random.randn(6,2),
                  index = dates,
                  columns = ['col1','col2'])

In [ ]:
df

La plupart des DataFrame que vous utiliserez seront lus au sein de fichier extérieurs (e.g : csv) 

La méthode **read_csv** de Pandas a différentes options : 

- **index_col** permet de préciser la colonne à prendre comme Index
- **parse_dates** est un booléen permettant de parser les chaines en dates automatiquement

Prenons l'exemple suivant : 

In [8]:
stock = pd.read_csv("data/stock_px.csv")

L'index de stock dans ce cas là est une liste d'Integer :

In [10]:
stock.head()

,date,AAPL,MSFT,XOM,SPX
0,2003-01-02 00:00:00,7.40,21.11,29.22,909.03
1,2003-01-03 00:00:00,7.45,21.14,29.24,908.59
2,2003-01-06 00:00:00,7.45,21.52,29.96,929.01
3,2003-01-07 00:00:00,7.43,21.93,28.95,922.93
4,2003-01-08 00:00:00,7.28,21.31,28.83,909.93


En spécifiant l'index à utiliser et en activant l'option de parsing de dates, nous obtenons : 

In [11]:
stock = pd.read_csv("data/stock_px.csv", parse_dates=True, index_col=0)

In [12]:
stock.index

<class 'pandas.tseries.index.DatetimeIndex'>
[2003-01-02, ..., 2011-10-14]
Length: 2214, Freq: None, Timezone: None

Dans le cas où il n'est pas possible de fixer un Index à la lecture d'un fichier, nous pourrons utiliser la méthode **set_index** pour fixer l'index d'un DataFrame après sa création

Lorsqu'un DataFrame a été correctement créé, Pandas offre de nombreuses méthodes pour sélectionner ou filtrer des données. 

Nous pouvons sélectionner par exemple uniquement les données de stock de l'année 2005 avec la clé '2005' :

In [14]:
stock['2005'].head()

,AAPL,MSFT,XOM,SPX
date,,,,
2005-01-03,31.65,23.70,43.45,1202.08
2005-01-04,31.97,23.79,43.15,1188.05
2005-01-05,32.25,23.73,42.93,1183.74
2005-01-06,32.28,23.71,43.47,1187.89
2005-01-07,34.62,23.64,43.19,1186.19


On peut de la même manière séléctionner un mois, ou une période :

In [15]:
stock['2005-03'].head()

,AAPL,MSFT,XOM,SPX
date,,,,
2005-03-01,44.50,22.47,54.13,1210.41
2005-03-02,44.12,22.45,54.63,1210.08
2005-03-03,41.79,22.37,54.96,1210.47
2005-03-04,42.81,22.37,55.41,1222.12
2005-03-07,42.75,22.64,54.91,1225.31


In [16]:
stock['2005-03':'2005-04'].head()

,AAPL,MSFT,XOM,SPX
date,,,,
2005-03-01,44.50,22.47,54.13,1210.41
2005-03-02,44.12,22.45,54.63,1210.08
2005-03-03,41.79,22.37,54.96,1210.47
2005-03-04,42.81,22.37,55.41,1222.12
2005-03-07,42.75,22.64,54.91,1225.31


## Resampling

Le **resampling** est l'art d'aggréger ou de désagréger des données temporelles. 

On peut vouloir par exemple aggréger les données du DataFrame stock en calculat la moyenne par année. 

On utilisera pour cela la méthode **resample** en spécifiant  **la fréquence** et la **fonction d'aggrégation** via le paramètre **how**. 

Un détail des valeurs possibles pour chacun de ces paramètres est disponible ici - http://pandas.pydata.org/pandas-docs/dev/timeseries.html#offset-aliases

In [ ]:
stock.resample('AS', how='mean')

## Plotting

La visualisation de données appliquée à des time series permet d'apprécier rapidement une tendance. 

Pandas permet de créer rapidement une visualisation avec la méthode **plot**

In [ ]:
# permet d'afficher les graphes au sein de la page

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
stock.plot()

In [ ]:
stock.resample('M', how='mean').plot()